# 1. Query Rewrite
透過將問題 (query) 改寫，提升文件抽取 (document retrieval) 的性能

## 1.1 MultiQuery
面對下面兩個痛點：
- 人類看起來 query 中的一小點措辭的改變，也可能會影響文件抽取的結果，導致需要嘗試不同的 query (prompt tuning)
- embeddings 沒辦法完整地捕捉所有 query 中的意思

我們可以嘗試:
1. 根據使用者的 query，產生多個關注不同面向的替代 query
2. 依據各個 query 獨立抽取文件
3. 將各個 query 抽取出的文件集合併並去重複 (unique union)

In [1]:
import logging
from langchain.vectorstores import Qdrant
from langchain.document_loaders import NotionDirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_setup import ChatOpenAI, pprint_documents, tracing_v2_enabled_if_api_key_set

# 讀取資料
loader = NotionDirectoryLoader("../../data/notion/")
documents = loader.load()

# 分割文件 (Document)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(documents)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Qdrant.from_documents(documents=splits, embedding=embedding, location=":memory:")

# 建立 MultiQueryRetriever
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(search_kwargs={'k': 2}), llm=llm
)

# 紀錄 (logging) 內部產生的問題 (queries)
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [2]:
with tracing_v2_enabled_if_api_key_set(project_name='tutorial'):
    question = "什麼是隕石落下式開發法?"
    unique_docs = retriever_from_llm.get_relevant_documents(query=question)
    pprint_documents(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 隕石落下式開發法的定義是什麼？', '2. 隕石落下式開發法有哪些特點？', '3. 隕石落下式開發法如何應用在實際開發中？']


Document 1:

今天來介紹日本最具代表性的軟體開發手法
它的名子為 **隕石落下型開發**。

# 第一節

通常的**瀑布式開發**是像下面這樣的形式:
| 步驟 | 內容     | 負責人     |
| ---- | -------- | ---------- |
| 1    | 要件定義 | Producer   |
| 2    | 基本設計 | Director   |
| 3    | 詳細設計 | Planner    |
| 4    | 實裝     | Programmer |

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md'}
----------------------------------------------------------------------------------------------------
Document 2:

而**隕石式開發**是像下面這樣子的形式：
|     | 步驟 | 內容     | 負責人     |
| --- | ---- | -------- | ---------- |
| 神  | 1    | 要件定義 | Producer   |
| 神  | 2    | 基本設計 | Director   |
| 神  | 3    | 詳細設計 | Planner    |
| 神  | 4    | 實裝     | Programmer |

然後就會這樣（全部都被隕石砸到爆炸）：

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md'}
----------------------------------------------------------------------------------------------------
Document 3:

然後就會這樣（全部都被隕石砸到爆炸）：

💥要件定義💥Producer💥基本設計💥Director💥詳細設計💥Planner💥實裝💥Programmer

這是敏捷式開法守法的循環

[要件定義->基本設計->詳細設計->實裝]->[要件定義->基本設計->詳細設計->實裝]->

但在

## 1.2 Self-querying

當遇到使用者問題 (query) 隱含了針對詮釋資料 (metadata) 的條件時

In [3]:
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain.chains.query_constructor.base import (
    load_query_constructor_chain,
    DEFAULT_EXAMPLES
)

from langchain_setup import OpenAI, pprint_documents, tracing_v2_enabled_if_api_key_set

# 文件 (documents)
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "rating": 9.9,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
            "rating": 9.9,
        },
    ),
]

# 建立 Vectorstore
embeddings = OpenAIEmbeddings()
vectorstore = Qdrant.from_documents(docs, embeddings, location=":memory:")

# This example specifies a query and composite filter
query = "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"

In [4]:
# 定義詮釋資料 (metadata)的每一個欄位 (field)
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

# 建立 SelfQueryRetriever
llm = OpenAI(temperature=0)
document_content_description = "Brief summary of a movie"
selfq_retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

# 實際抽取 (retrieve) 看看
with tracing_v2_enabled_if_api_key_set(project_name='tutorial'):
    selfq_retrieved_documents = selfq_retriever.get_relevant_documents(query)
pprint_documents(selfq_retrieved_documents)

[LangSmith URL]: https://smith.langchain.com/o/34ec837d-8405-462d-b949-fdfaebda792b/projects/p/fdcbda35-4d3a-418b-ab49-7e3205e630a6/r/acd95022-aca8-4dc9-93a4-964ba5c59103?poll=true
Document 1:

Toys come alive and have a blast doing so

Metadata:{'genre': 'animated', 'year': 1995}


這功能由一個複雜的 prompt 來實現，這個 prompt 依序提供了大語言模型 (LLM) 以下的資訊
- 目標：將使用者問題結構化來對齊提供的資料格式
- 輸出格式說明 (formatting instruction)
- 解釋有哪些比較計算子 (comparison operators) 如等於、大於，哪些邏輯運算子 (logical operators) 如 and、or 可以用，還有怎麼使用
- 數個範例，每個範例包含
    - 資料內容的簡短說明
    - 詮釋資料 (metadata) 欄位 (field) 的介紹
    - 使用者問題 (user query)
    - 模型產出的結構化的回答

而建構這個複雜的 prompt ，除了我們提供的文件內容 (document content) 的說明 `document_content_description` 和詮釋資料 (metadata) 的說明 `metadata_field_info`，Langchain 自動提供了範例和能配合 vector store 的運算子 (operators) 的資訊。

In [5]:
chain = load_query_constructor_chain(
    llm=llm,
    document_contents=document_content_description,
    attribute_info=metadata_field_info,
    # 例子是 Langchain 預先提供的
    examples=DEFAULT_EXAMPLES,
    # 使用所選擇的 vectoreDB 所支援的，詮釋資料 (metatdat) 篩選 (filter) 用的運算子 (operators)
    allowed_comparators=ChromaTranslator.allowed_operators,
    allowed_operators=ChromaTranslator.allowed_operators,
)
print(chain.prompt.format(query=query))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (and | or): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or): logical operator
- `statement1`, `statement2`, ... (

# MultiVector
我們想要將文件 (document) 的內容完整地傳給大語言模型 (LLM)，但是這個文件內容因為某些原因（例如包含的資訊太多太雜）而不利於 embedding-based retrieval。

或是不想更改文件本身內容，但又想提升文件抽取 (document retrieval) 的準度的時候可以怎麼辦？

Langchain 實作並介紹了以下三種作法
- 分塊 (chunking)
- 摘要 (summarization): 對文件進行摘要，並將摘要的 embedding 和/取代 文件的 embedding 作為該文件的對應
- 虛擬命令/問題 (hypothetical queries): 生成適合該文件回答的問題，並將該問題的 embedding 或該問題加文件的 embedding 作為對應。

這些作法的共通思維是：針對源文件(source document)衍生出能夠對應到該文件的某些衍生物(derivatives)，而這些衍生物會取代或跟著源文件一起成為抽取的候選，若選到某個衍生物，則會追循其對應抽取出其對應的整份源文件。

## Chunking
太長或太雜的文件會導致細部語義會彼此稀釋。

In [1]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.schema import Document

from langchain_setup import pprint_documents, pprint
from langchain_setup.qdrant import pprint_qdrant_documents

# 文件
doc = """\
人口密度大的地鐵里，空氣中的至少有15%包含著每個人的皮膚。

其實貓咪其實也和人類一樣，是分左撇子和右撇子的，不過兩者數量差不多，大家可以關注一下身邊的喵星人，可能會發現它們習慣用哪一隻爪子，就知道它們到底是左撇子還是右撇子了。

-----

某大學的研究室發現，世界上至少有16%部手機上是沾有糞便等排洩物的。

原子如果沒有了空隙的話，那麼全世界的人類可能會被擠壓到蘋果那麼大的空間。
"""
documents = [Document(page_content=doc)]

# 切割文件 (document splitting)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=0, separators=["-----"], keep_separator=False
)
_documents = splitter.split_documents(documents)

# 建立 retriever
retriever = Qdrant.from_documents(
    _documents, OpenAIEmbeddings(), location=":memory:"
).as_retriever(search_kwargs={"k": 1})
pprint_documents(retriever.get_relevant_documents("貓咪被擠壓到蘋果大的空間時會用哪隻手清理糞便等排泄物？"))

Document 1:

某大學的研究室發現，世界上至少有16%部手機上是沾有糞便等排洩物的。

原子如果沒有了空隙的話，那麼全世界的人類可能會被擠壓到蘋果那麼大的空間。

Metadata:{}


切分文件 (parent document) 並讓各切分 (child document) 的 embedding 也都對應到該文件 (parent document)。

使得文件能夠對應到更細部的語義，但同時又能夠對應到整份文件而非部分文件給下一步驟。

In [2]:
# 親分割器 (parent splitter) 負責從原生文件 (raw documents) 切出親文件 (parent documents)
# parent documents 是我們要傳給大語言模型 (LLM) 的
# parent splitter 最重要的任務之一是符合大語言模型的可處理長度或可有效處理的長度
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=0, separators=["-----"], keep_separator=False
)

# 子分割器 (child splitter) 負責再從每個親文件中各自切出更小的子文件 (child documents)
# 子分割器的任務是要切出適合 embedding-based retrieval 的片段
# 通常會希望該片段內同質性高少雜訊或不會太長
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50, chunk_overlap=0, separators=["\n\n"], keep_separator=False
)

# Empty Vectore store
vectorstore = Qdrant.from_texts(
    ["dummy"], embedding=OpenAIEmbeddings(), location=":memory:"
)  #
vectorstore.delete(
    [vectorstore.client.scroll(vectorstore.collection_name)[0][0].id]
)  # delete dummy

# ParentDocumentRetriever
chunk_retriever = ParentDocumentRetriever(
    # vectore 會拿來儲存子文件和其 embeddings
    vectorstore=vectorstore,
    # docstore 會儲存親文件
    docstore=InMemoryStore(),
    # 把 splitter 給 retriever 讓 retriever 在新增文件時幫忙切
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,  # 不給也是可以的，會直接把傳入的文件 (documents) 做為親文件 (parent documents)
    # 只取最相關的一個文件
    search_kwargs={"k": 1},
)

# 實際看看這樣細分的方法能不能幫助我們抽取到正確相關的文件
chunk_retriever.add_documents(
    documents
)  # 把原生文件 (raw documents) 丟進去給 `ParentDocumentRetriever` 切分和處理
retrieved_documents = chunk_retriever.get_relevant_documents("貓咪被擠壓到蘋果大的空間時會用哪隻手清理糞便等排泄物？")
pprint_documents(retrieved_documents)

Document 1:

人口密度大的地鐵里，空氣中的至少有15%包含著每個人的皮膚。

其實貓咪其實也和人類一樣，是分左撇子和右撇子的，不過兩者數量差不多，大家可以關注一下身邊的喵星人，可能會發現它們習慣用哪一隻爪子，就知道它們到底是左撇子還是右撇子了。



Metadata:{}


我們可以看到 vectorstore 儲存了子文件 (child document) 對應的親文件 (parent document) 的識別碼 (id)

而 docstore 則儲存親文件 (parent documents) 和其識別碼 (ids)

In [3]:
print("VECTOR STORE", end="\n\n")
pprint_qdrant_documents(vectorstore)
print("DOCSTORE", end="\n\n")
pprint(chunk_retriever.docstore.__dict__)

VECTOR STORE

Document 63d5171a5e8e4d2db8f93ab8b36a9bdf:

原子如果沒有了空隙的話，那麼全世界的人類可能會被擠壓到蘋果那麼大的空間。

Metadata:{'doc_id': 'f63569e1-a7a8-40ad-aaac-fe3116edce4b'}
----------------------------------------------------------------------------------------------------
Document 6849b0c9f22f421dbe8f1483bc3298a6:

某大學的研究室發現，世界上至少有16%部手機上是沾有糞便等排洩物的。

Metadata:{'doc_id': 'f63569e1-a7a8-40ad-aaac-fe3116edce4b'}
----------------------------------------------------------------------------------------------------
Document 7424ec0f04f146dba2edc81f6e81edc7:

人口密度大的地鐵里，空氣中的至少有15%包含著每個人的皮膚。

Metadata:{'doc_id': '95c8b6fb-5154-46f8-8138-5c9fdb777b6b'}
----------------------------------------------------------------------------------------------------
Document bc112bc738204db4a62b49adea73ee50:

其實貓咪其實也和人類一樣，是分左撇子和右撇子的，不過兩者數量差不多，大家可以關注一下身邊的喵星人，可能會發現它們習慣用哪一隻爪子，就知道它們到底是左撇子還是右撇子了。

Metadata:{'doc_id': '95c8b6fb-5154-46f8-8138-5c9fdb777b6b'}
DOCSTORE

{'store': {'95c8b6fb-5154-46f8-8138-5c9fdb777b6b': Document(page_c

## Summary
當文章本身非常長或資訊非常雜，embedding 會雜質很多。

In [1]:
from operator import attrgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.document import Document
from langchain.vectorstores import Qdrant
from langchain.storage import InMemoryStore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import MultiVectorRetriever
from langchain_setup import ChatOpenAI, pprint_documents

documents = [
    Document(
        page_content="比黑色更黑，比黑暗更暗的漆黑，在此寄託吾真紅的金光吧！覺醒之時的到來，荒謬至極的墮落章理，成為無形的扭曲而顯現吧！起舞吧，起舞吧，起舞吧！吾之力量本源之愿的崩壞，無人可及的崩壞，將天地萬象焚燒殆盡，自深淵降臨吧，這就是人類最強威力的攻擊手段，這就是究極攻擊魔法，Explosion!"
    )
]

我們可以透過重點整理來精煉 embedding，使 embedding-based retrieval 更準

In [2]:
# 從源文件 (source documents) 產生衍生物 (derivatives)
chain = (
    {"doc": attrgetter("page_content")}
    | ChatPromptTemplate.from_template("請用一句話摘要下列的資訊:\n\n{doc}")
    | ChatOpenAI()
    | StrOutputParser()
)
summaries = chain.batch(documents)  # 平行化處理

# 標注源文件和衍生物的對應
child_documents = []
document_ids = []
for i, (summary, document) in enumerate(zip(summaries, documents)):
    document_id = str(i)
    child_document = Document(page_content=summary, metadata={"doc_id": document_id})
    child_documents.append(child_document)
    document_ids.append(document_id)

# 衍生物存入 vectorestore
vectorstore = Qdrant.from_documents(
    documents=child_documents, embedding=OpenAIEmbeddings(), location=":memory:"
)

# [Optional] 將源文件也存入 vectorstore
_documents = []
for document_id, document in zip(document_ids, documents):
    _document = Document(
        page_content=document.page_content, metadata={"doc_id": document_id}
    )
    _documents.append(_document)
vectorstore.add_documents(documents=_documents)

# 儲存源文件 (source documents) 及其識別 (id)
docstore = InMemoryStore()
docstore.mset(list(zip(document_ids, documents)))

# 建立 MultiVectorRetriever
summary_retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key="doc_id",
)

# 試著抽取看看
retrieved_docs = summary_retriever.get_relevant_documents("什麼魔法會將一切燃燒殆盡？")
pprint_documents(retrieved_docs)

Document 1:

比黑色更黑，比黑暗更暗的漆黑，在此寄託吾真紅的金光吧！覺醒之時的到來，荒謬至極的墮落章理，成為無形的扭曲而顯現吧！起舞吧，起舞吧，起舞吧！吾之力量本源之愿的崩壞，無人可及的崩壞，將天地萬象焚燒殆盡，自深淵降臨吧，這就是人類最強威力的攻擊手段，這就是究極攻擊魔法，Explosion!

Metadata:{}


其實真正透過 embedding-based retrieval 選到的是其衍生物

In [3]:
summary_retriever.vectorstore.similarity_search(query="什麼魔法會將一切焚燒殆盡？", k=1)

[Document(page_content='使用最強的攻擊魔法"Explosion"，將一切燃燒殆盡。', metadata={'doc_id': '0'})]

## Hypothetical Queries
當我們在找資料時會看有沒有其他人問過類似的問題，而在文件抽取（document retrieval）中我們也可以實踐這個思路

In [1]:
from operator import attrgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.document import Document
from langchain.vectorstores import Qdrant
from langchain.storage import InMemoryStore
from langchain.retrievers import MultiVectorRetriever
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.chains.openai_functions.base import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel
from langchain.embeddings import OpenAIEmbeddings
from langchain_setup import ChatOpenAI, pprint_documents

documents = [
    Document(
        page_content="比黑色更黑，比黑暗更暗的漆黑，在此寄託吾真紅的金光吧！覺醒之時的到來，荒謬至極的墮落章理，成為無形的扭曲而顯現吧！起舞吧，起舞吧，起舞吧！吾之力量本源之愿的崩壞，無人可及的崩壞，將天地萬象焚燒殆盡，自深淵降臨吧，這就是人類最強威力的攻擊手段，這就是究極攻擊魔法，Explosion!"
    )
]

我們讓大語言模型 (LLM) 根據文件 (document) 內容自動產生問句 (queries)，以這些問句作為該文件的衍生物 (derivatives)，抽取到這些問題就回傳其對應的源文件 (source document)。這個方法有幾個好處
1. 每個問題可能可以代表內容不同的面向或措辭
2. 除了自動產生的問題外，也可以手動將失敗的問題 (query) 對應到正確的文件

In [2]:
# 從源文件 (source documents) 產生衍生物 (derivatives)
class hypothetical_questions(BaseModel):
    """Generate hypothetical questions"""

    questions: list[str]


function = convert_to_openai_function(hypothetical_questions)

query_gen_chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("請產生三個可以根據以下的資訊回答的問題:\n\n{doc}")  # 可以設定更多個
    # 在這邊為了教學將溫度 (temperature) 設為零，但實際上增加一些隨機性會比較好
    | ChatOpenAI(temperature=0).bind(
        functions=[function], function_call={"name": "hypothetical_questions"}
    )
    | JsonKeyOutputFunctionsParser(key_name="questions")
)
questions = query_gen_chain.batch(documents)

# 標注源文件和衍生物的對應
child_documents = []
document_ids = []
for i, (_questions, document) in enumerate(zip(questions, documents)):
    document_id = str(i)
    for question in _questions:
        child_document = Document(
            page_content=question, metadata={"doc_id": document_id}
        )
        child_documents.append(child_document)
    document_ids.append(document_id)

# 衍生物存入 vectorestore
vectorstore = Qdrant.from_documents(
    documents=child_documents, embedding=OpenAIEmbeddings(), location=":memory:"
)

# [Optional] 將源文件也存入 vectorstore
_documents = []
for document_id, document in zip(document_ids, documents):
    _document = Document(
        page_content=document.page_content, metadata={"doc_id": document_id}
    )
    _documents.append(_document)
vectorstore.add_documents(documents=_documents)

# 儲存源文件 (source documents) 及其識別 (id)
docstore = store = InMemoryStore()
docstore.mset(list(zip(document_ids, documents)))

# 建立 MultiVectorRetriever
hypothetical_questions_retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key="doc_id",
)

# 試著抽取看看
hypothetical_questions_retriever.get_relevant_documents("什麼魔法是人類最強的攻擊？")

[Document(page_content='比黑色更黑，比黑暗更暗的漆黑，在此寄託吾真紅的金光吧！覺醒之時的到來，荒謬至極的墮落章理，成為無形的扭曲而顯現吧！起舞吧，起舞吧，起舞吧！吾之力量本源之愿的崩壞，無人可及的崩壞，將天地萬象焚燒殆盡，自深淵降臨吧，這就是人類最強威力的攻擊手段，這就是究極攻擊魔法，Explosion!')]

其實真正透過 embedding-based retrieval 選到的是其衍生物

In [3]:
print(questions)

hypothetical_questions_retriever.vectorstore.similarity_search(
    query="什麼魔法是人類最強的攻擊？", k=1
)

[['什麼顏色比黑色更黑？', '什麼比黑暗更暗？', '什麼是人類最強威力的攻擊手段？']]


[Document(page_content='什麼是人類最強威力的攻擊手段？', metadata={'doc_id': '0'})]

**想想看**: MultiQuery 是從使用者問句 (user query)，改寫成多個問句。Hypothetical queries 則是從文件出發，設計不同的問句。這兩個方法間在能達成的效果上有何不同？什麼情況下用哪個比較好？可以合併一起用嗎？
<details>
<summary>參考</summary>
當使用者問句非常複雜且隱含複數條件或要求時，可以利用MultiQuery 將其拆解成數個比較單純的 query。hypothetical queries 則可以當作是某種拆解複雜文件資訊成不同的單純面向的方法。另外個人認為兩種方法都可以達到減少因為措辭差異而對應不到的問題。兩者其實不是只能使用一種，合併使用是可能的。
</details>

# TimeWeightedVectorStoreRetriever

`recency_score = semantic_similarity + (1.0 - decay_rate) ^ hours_passed_since_last_access`
- 跟何時創建的檔案無關，就算是很以前的檔案，只要最近有被抽取到，就是新鮮的
- 會給每個文件自動標註上
  - `last_accessed_at` (最後存取時間): 用於計算經過的時間
  - `created_at` (創建時間): 沒有被用到，可能只是讓你參考用
- `decay_rate` 越大，越久沒被碰的文件越不容易被碰

In [1]:
import time
from datetime import datetime, timedelta
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_setup import pprint_documents
from langchain_setup.qdrant import create_inmemory_empty_qdrant, pprint_qdrant_documents

# 以空的 vector store 來建立 retriever
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=create_inmemory_empty_qdrant(), decay_rate=0.999, k=1
)

yesterday = datetime.now() - timedelta(days=1)
# 最後存取時間 (last_accessed_at) 和 創建時間 (created_at) 可以手動設定
retriever.add_documents(
    [Document(page_content="昨天的風兒真是喧囂呢", metadata={"last_accessed_at": yesterday})]
)
# 或是不設定的話就是兩個皆設為現在時間
retriever.add_documents([Document(page_content="這個味道是說謊的味道")])

# 檢視 vector store 內部
pprint_qdrant_documents(retriever.vectorstore)
time.sleep(1)

# 由於設定極高 decay rate ，抽取 (retrieve) 到過去取用 (access) 的文件的可能性極低
# 被抽取的文件的最後存取時間會自動更新
print("\n=========== 抽取 (retrieval) 結果 ===========\n")
pprint_documents(retriever.get_relevant_documents("風兒很喧囂"))

Document 08f4b87f627d43c6b6cc80a175712efd:

昨天的風兒真是喧囂呢

Metadata:
{'buffer_idx': 0,
 'created_at': datetime.datetime(2023, 11, 10, 18, 1, 27, 66934),
 'last_accessed_at': datetime.datetime(2023, 11, 9, 18, 1, 27, 66934)}
----------------------------------------------------------------------------------------------------
Document 6cc62d526e26428ab1ce29edf074fc4f:

這個味道是說謊的味道

Metadata:
{'buffer_idx': 1,
 'created_at': datetime.datetime(2023, 11, 10, 18, 1, 27, 313571),
 'last_accessed_at': datetime.datetime(2023, 11, 10, 18, 1, 27, 313571)}

=========== 抽取 (retrieval) 結果 ===========

Document 1:

這個味道是說謊的味道

Metadata:
{'buffer_idx': 1,
 'created_at': datetime.datetime(2023, 11, 10, 18, 1, 27, 313571),
 'last_accessed_at': datetime.datetime(2023, 11, 10, 18, 1, 28, 578207)}


想想看:
- 越久沒被碰的文件越被難抽取，在什麼樣的情況下會造成惡性循環?
- 要怎麼實作用「檔案創建時間」或「檔案內容更新時間」來做 time decay ?

<details>
<summary>參考</summary>
有用但是很少被用的文件，因少被抽取而被降低順序，因降低順位而被更少抽取

可以嘗試手動將文件 (Document) 的 'last_accessed_at' 值設成我們想要的日期時間
</details>

# Ensemble Retriever
在介紹抽取器 (Retriever) 時我們提到了除了基於 embeddings 相似度以外的抽取器，而我們剛才也介紹過了許多進階的抽取器。每個抽取器都有不同的特性，我們是否可以結合不同的優勢來截長補短？

## 並聯
將從不同種類的抽取器所得到的相關性分數 (relevance score) 以使用者設定的權重 (weights) 結合，作為該文件 (document) 的最終相關性分數，並以該分數作為根據抽取 (retrieve)。

這種方法雖然比較花成本，但是可能可以提高召回率 (recall)，也就是減少真的相關的文件 (document) 沒有被抽取到的機率。

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import Qdrant
from langchain_setup import pprint_documents

doc_list = [
    "Apple pen~",
    "Pineapple pen~",
    "Ugh! Pen Pineapple Apple Pen",
]

# 基於語面的抽取 (Lexical-based retrieval) (BM25)
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

# 基於語義 embedding 的抽取 (Embedding-based retrieval)
vectorstore = Qdrant.from_texts(doc_list, OpenAIEmbeddings(), location=":memory:")
embedding_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 組合 (ensemble) 兩種抽取器 (retriever)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, embedding_retriever], weights=[0.5, 0.5]
)

docs = ensemble_retriever.get_relevant_documents("apples")
pprint_documents(docs)

Document 1:

Ugh! Pen Pineapple Apple Pen

Metadata:{}
----------------------------------------------------------------------------------------------------
Document 2:

Apple pen~

Metadata:{}
----------------------------------------------------------------------------------------------------
Document 3:

Pineapple pen~

Metadata:{}


## 串連
多階段的（multi-staged）方法在建構搜尋系統中是常被使用到的方法。其會先用比較弱但比較快的排序 (ranking) 方法，配合比較寬鬆的相關度閥值 (threshold) 做大量篩選，再只把粗篩過後的結果傳給下一個更強但更慢的排序 (ranking) 方法做篩選或排序，然後可能再重複傳給下一個更強排序方法的過程。

當文本 (documents) 數量很大時，這是一個可以同時顧及成本、精準度 (precision)、召回率 (recall) 的方法

雖然 Langchain 沒有介紹串連的作法或現成的實作，但我們還是可以透過現有的東西拼湊出來

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain_setup import pprint_documents
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers import ContextualCompressionRetriever
from langchain_setup import ChatOpenAI

doc_list = [
    "天之鎖",
    "修爾夏伽那",
    "天地乖離開闢之星",
    "王之號砲",
    "黑帝斯的隱形頭盔",
    "Apple pen~",
    "Pineapple pen~",
    "Ugh! Pen Pineapple Apple Pen",
]

# 基於語義 embedding 的抽取 (Embedding-based retrieval) 配上一個相對寬鬆的篩選閥值 (threshold)
vectorstore = Qdrant.from_texts(doc_list, OpenAIEmbeddings(), location=":memory:")
embedding_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.75}
)

# 以大型語言模型 (LLM) 來直接同時比較兩篇文章的方法
stronger_ranking = LLMChainFilter.from_llm(ChatOpenAI(temperature=0))

# 串連
cascade_retriever = ContextualCompressionRetriever(
    base_retriever=embedding_retriever, base_compressor=stronger_ranking
)

docs = cascade_retriever.get_relevant_documents("apples")
pprint_documents(docs)

c:\Users\121664\micromamba\envs\dev\lib\site-packages\langchain\chains\llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\121664\micromamba\envs\dev\lib\site-packages\langchain\chains\llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\121664\micromamba\envs\dev\lib\site-packages\langchain\chains\llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

Apple pen~

Metadata:{}
----------------------------------------------------------------------------------------------------
Document 2:

Ugh! Pen Pineapple Apple Pen

Metadata:{}


從下方例子可以看出，在第一階段時便已過濾掉明顯不相關的文件 (document)

In [2]:
embedding_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.75}
)
docs_after_1st_stage = embedding_retriever.get_relevant_documents('apples')
pprint_documents(docs_after_1st_stage)

Document 1:

Apple pen~

Metadata:{}
----------------------------------------------------------------------------------------------------
Document 2:

Ugh! Pen Pineapple Apple Pen

Metadata:{}
----------------------------------------------------------------------------------------------------
Document 3:

Pineapple pen~

Metadata:{}


**想想看** 偏早期階段 (stage) 排序 (ranking) 法跟偏後期的階段的排序法，目標是怎麼的不一樣？
<details>
<summary>參考</summary>
前面的方法側重於在不要誤過濾掉相關文件的前提下 (重視 Recall)，如何快速而低成本過濾掉大部分的文件，減少傳給後面高成本排序法的文件的數量。越靠後面的方法則會越側重於給出準確的相關度排序 (重視 Precision)，只留下真的相關的文件或給出非常好的相關度排序。
</details>